## 0. Import Libraries

In [1]:
import numpy as np
import xarray as xr

## 1. Open Data

### 1.1 Data Options

In [61]:
# change these
name_dt = 'cosmo' # cosmo or era5land
season = 'SON' # DJF, MAM, JJA, SON
time_resample = 6 # options: 1, 3, 6 (resampling)

# path for resampling information
if time_resample != 1:
    path_resample = fr'{time_resample}hrsum'
else:
    path_resample = fr'{time_resample}hr'
    
# general data path
path_general = fr'/mnt/e/JupyterLab/Yuksek_Lisans/msc_thesis_data/'

# variable name
name_variable = 'TOT_PREC'

# saving path
path_save = fr'data/{name_variable}/{name_dt}/'

In [62]:
# open data
path_dt = fr'{path_general}Tr011_MPI_RF_PR_{season}_{path_resample}_198501_200512.nc'

# chunk recommendation from chatGPT
dt = xr.open_dataset(path_dt, 
                     chunks={'time': 10,
                             })[name_variable]

In [63]:
dt

<xarray.DataArray 'TOT_PREC' (time: 29584, rlat: 107, rlon: 175)>
dask.array<open_dataset-9d1335916811e5ceede306aa89d275baTOT_PREC, shape=(29584, 107, 175), dtype=float32, chunksize=(10, 107, 175), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1985-09-01 ... 2005-11-30T18:00:00
    lon      (rlat, rlon) float32 dask.array<chunksize=(107, 175), meta=np.ndarray>
    lat      (rlat, rlon) float32 dask.array<chunksize=(107, 175), meta=np.ndarray>
  * rlon     (rlon) float32 -8.3 -8.19 -8.08 -7.97 ... 10.51 10.62 10.73 10.84
  * rlat     (rlat) float32 -6.2 -6.09 -5.98 -5.87 -5.76 ... 5.13 5.24 5.35 5.46

## 2. Intensity Indices

In [64]:
# groupby and find maximum (rx_xhr)
rx_xhr = dt.groupby('time.year').reduce(np.max, 
                                        dim='time', 
                                        keep_attrs=True
                                       )

# Add information about the computation to the dataset's attributes
rx_xhr.attrs['computation'] = 'grouped by year and took maximum over time dimension'
rx_xhr.attrs['indice_name'] = fr'rx{time_resample}hr'

In [65]:
rx_xhr.to_netcdf(fr"{path_save}{name_dt}_{season}_intensity_{rx_xhr.attrs['indice_name']}_198501_200512.nc")